<a href="https://colab.research.google.com/github/Johyeonje/DeepLearningStudy/blob/master/Start!_chap7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import Image
import tensorflow as tf
import numpy as np

#순환 신경망

: **순서가 있는 데이터**를 입력으로 받고, 같은 네트워크를 이용해 변화하는 입력에 대한 출력을 얻어낸다. `데이터의 되먹임 구조`    
순서가 있는 데이터란 자연어, 음악, 날씨, 주가 등 시간의 흐름에 따라 변화하고 변화가 의미를 갖는 데이터

##?
1. 전 입력에 대한 출력을 나중 입력과 합쳐서한다는건가
2. 무조건 합쳐야한 나중 입력은 들어올 수가 있는가
3. 입력과 출력의 사이즈가 다를 수 있는데 이건 어쩌나


답 :
1. 전 입력에 대한 출력과 나중의 입력을 합쳐서 계산하는것이 맞음
2. 무조건 합치는 것이고 합칠 때 일련의 연산이 있음.
3. 같게하는 것임.

##순환 신경망의 구조

$X_1,X_2,X_3$으로 변할 때 같은 네트워크를 사용해 출력인 $Y_1,Y_2,Y_3$를 반환하고 있음을 알 수 있다.    
중요한 점은 `출력값이 다음 입력을 받을 때의 RNN 네트워크에도 동일하게 전달` 되고 있다는 것이다.

In [ ]:
Image('/content/drive/My Drive/DeepLearning/Start!_Tensorflow/images/RNN.png', width=600)

순환 신경망은 입력과 출력의 길이 제한이 없기 때문에 아래와 같은 다양한 네트워크의 구성이 가능하다.

In [ ]:
Image('/content/drive/My Drive/DeepLearning/Start!_Tensorflow/images/RNN2.png', width=600)

##주요 레이어 정리

###SimpleRNN 레이어

In [ ]:
Image('/content/drive/My Drive/DeepLearning/Start!_Tensorflow/images/simpleRNN.png', width=400)

가장 간단한 형태의 RNN 레이어이다. $x_{t-1}, x_t$ 등은 SimpleRNN에 들어가는 입력을 나타내고, $h_{t-1}, h_t$ 등은 SimpleRNN 레이어의 출력을 나타낸다. $U$와 $W$는 입력과 출력에 곱해지는 가중치 이다.

SimpleRNN 레이어의 출력의 수식은 다음과 같다.

$h_t = tanh(U_{x_t} + Wh_{t-1})$

활설화함수로는 tanh가 쓰인다. 앞에 나왔던 것처럼 tanh는 실수를 입력받아 -1에서 1 사이의 출력 값을 반환하는 활성화 함수이다.   
ReLU나 다른 것으로 대체될 수 있다.

####SimpleRNN 레이어 생성 코드

In [ ]:
rnn1 = tf.keras.layers.SimpleRNN(units=1, activation='tanh', return_sequences=True)

units : SimpleRNN 레이어에 존재하는 뉴런의 수이다.    
return_sequences : 출력으로 시퀀스 전체를 출력할지 여부를 나타내는 옵션으로 주로 여러 개의 RNN 레이어를 쌓을 때 쓰인다.

####시퀀스 예측 모델 예제

시퀀스를 구성하는 앞쪽 4개의 숫자가 주어졌을 때 그 다음에 올 숫자를 예측하는 간단한 "시퀀스 예측 모델"을 만들기 위해 SimpleRNN 레이어를 사용해보자.    
정해진 길이의 시퀀스를 주고 예측하는 것은 Dense만을 가지고 가능하지만 SimpleRNN으로 해보자.

#####학습 데이터 생성

In [ ]:
X = []
Y = []
for i in range(6):
  # [0,1,2,3], [1,2,3,4] 같은 정수의 시퀀스를 만든다.
  lst = list(range(i, i+4))

  # 위에서 구한 시퀀스의 숫자들을 각각 10으로 나눈 다음 저장한다.
  # SimpleRNN에 각 타임스텝에 하나씩 숫자가 들어가기 때문에 여기서도 하나씩 분리해서 배열에 저장한다.
  X.append(list(map(lambda c: [c/10], lst)))

  # 정답에 해당하는 4, 5 등의 정수 역시 앞에서처럼 10으로 나눠서 저장한다.
  Y.append((i+4)/10)

X = np.array(X)
Y = np.array(Y)
for i in range(len(X)):
  print(X[i], Y[i])

#####네트워크 정의

In [ ]:
model = tf.keras.Sequential([
        tf.keras.layers.SimpleRNN(units=10, return_sequences=False, input_shape=[4,1]),
        tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

중요한 점은 input_shape이다. 여기서 [4, 1]은 각각 timesteps, input_dim을 나타낸다.    
`timesteps` : 타입스텝이란 순환 신경망이 입력에 대해 계산을 반복하는 횟수    
`input_dim` : 입력 벡터의 크기

In [ ]:
print(X.shape)

X가 [4,1] 차원의 벡터가 6개 쌓여 생성된 것이므로 타임 스텝이 4, input_dim은 1 이된다.

In [ ]:
Image('/content/drive/My Drive/DeepLearning/Start!_Tensorflow/images/Seq_model.png', width=400)

시퀀스 예측 모델은 4 타임스텝에 걸쳐 입력을 받고, 마지막에 출력값을 다음 레이어로 반환한다.    
아까 추가한 Dense 레이어는 별도의 활성화함수가 없기 때문에 $h_3$는 바로 $y_3$가 된다.    
그리고 이값이 Label로 제시된 0.4와의 차이가 mse(Mean Squared Error), 즉 평균 제곱 오차가 된다.

In [ ]:
model.fit(X, Y, epochs=100, verbose=0)
print(model.predict(X))

얼추 비슷하게 예측한 것으로 보이는데 학습한 적 없던 데이터는 넣으면 어떻게 될까

In [ ]:
print(model.predict(np.array([[[0.6],[0.7],[0.8],[0.9]]])))
print(model.predict(np.array([[[-0.1],[0.0],[0.1],[0.2]]])))

더 좋은 결과를 받으려면 훈련데이터를 증량하는 것이 좋을 것으로 보인다.    
하지만 시퀀스를 정확하게 예측하기는 쉽지가 않다.

###LSTM 레이어

**SimpleRNN 레이어에는 한 가지 치명적인 단점**이 있는데, 바로 입력 데이터가 길어질수록, 즉 `데이터의 타입스텝이 커질수록 학습 능력이 떨어진다.` 라는 점이다.    
이를 `장기의존성(Long-Term-Dependency) 문제` 라고하며, `입력 데이터와 출력 사이의 길이가 멀어질수록 연관 관계가 적어진다`는 것이다.

이 문제를 해결하기 위해 셉 호흐라이터(Sepp Hochreiter)와 유르겐 슈미트후버에 의해 `LSTM(Long Short Term Memory)`가 1997년 제안되었다.    
LSTM은 RNN에 비해 복잡한 구조인데, 가장 큰 특징은 `출력 외에 LSTM 셀 사이에서만 공유되는 셀 상태(cell state)를 가진다`는 것이다.

In [ ]:
Image('/content/drive/My Drive/DeepLearning/Start!_Tensorflow/images/cmp_RNN_LSTM.png', width=700)